In [ ]:
# Environment Variables.
import os
# GUI Packages.
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets
import time
import hashlib
from datetime import datetime
import csv

In [ ]:
# TODO: Uncomment to install boto3 in the Colab environment.
!pip install boto3

# AWS Packages.
import boto3

     |████████████████████████████████| 131 kB 5.0 MB/s 
     |████████████████████████████████| 79 kB 2.8 MB/s 
     |████████████████████████████████| 8.5 MB 23.0 MB/s 
     |████████████████████████████████| 138 kB 47.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# TODO: Set the variable values with appropriate values.
accessKeyID = "..."
secretAccessKey = "..."
s3BucketName = "..."
mainInputImageFileName = "userInput.csv" # user input for prediction
mainresultsDataFileName = "result.txt" # output predicted by model
folderName = "/content"
inputDataFileName = "newData.csv" # manager input

## Pipeline to upload images and then parse results



In [ ]:
# modify this to match real file name
def parseAndShowResults(resultsDataFileName):
    with open(resultsDataFileName, "r") as results:
        lines = results.readlines()
        display(lines)
        results.close()

In [ ]:
global HID
def hashID(HID):
  HID = str(HID)
  # Create an instance of S3 with "accessKeyID" and "secretAccessKey" and replace "None" with it.
  client = boto3.client(
      's3',
      aws_access_key_id=accessKeyID,
      aws_secret_access_key=secretAccessKey
  )
  # Waiting and checking to see if the results.txt has been produced and placed in S3 from Lambda.
  while True:
      try:
          # Use the client to download the results.txt from S3 to the local directory.
          client.download_file(s3BucketName, HID, HID)
          print("Downloading")
          break
      except:
          print("Waiting for result")
          time.sleep(awsProgressRefreshRateSlider.value)

  parseAndShowResults(HID)

In [ ]:
 ## AWS Image Upload callback function and button ##

# Upload test.csv to S3 to produce the results.txt using lambda.
def awsImageUpload(buttonObject):
    
    # Create an instance of S3 with "accessKeyID" and "secretAccessKey" and replace "None" with it.
    client = boto3.client(
        's3',
        aws_access_key_id=accessKeyID,
        aws_secret_access_key=secretAccessKey
    )
    # path: "/content/test.csv"
    # Use the client to upload test.csv to S3.
    client.upload_file(str(folderName + "/" + inputImageFileName), s3BucketName, inputImageFileName)
    

    if len(hashKeyList) < 2:
      # Waiting and checking to see if the results.txt has been produced and placed in S3 from Lambda.
      while True:
          try:
              # Use the client to download the results.txt from S3 to the local directory.
              client.download_file(s3BucketName, resultsDataFileName, resultsDataFileName)
              print("Downloading")
              break
          except:
              print("Waiting for result")
              time.sleep(awsProgressRefreshRateSlider.value)

      # Use the client to remove test.csv and results.txt from S3.
      client.delete_object(Bucket=s3BucketName,Key=inputImageFileName)

      parseAndShowResults(resultsDataFileName)
    else:
      dropdownID = widgets.interact(hashID, HID = widgets.Dropdown(options = sorted(hashKeyList), description='HashID:'))
      display(dropdownID)
    

In [ ]:
def awsDataUpload(buttonObject):
  # Create an instance of S3 with "accessKeyID" and "secretAccessKey" and replace "None" with it.
    client = boto3.client(
        's3',
        aws_access_key_id=accessKeyID,
        aws_secret_access_key=secretAccessKey
    )
    # path: "/content/test.csv"
    # Use the client to upload test.csv to S3.
    client.upload_file(str(folderName + "/" + inputDataFileName), s3BucketName, inputDataFileName)

In [ ]:
## File upload callback function and button ##
def selectimage2upload(imageData): 
    # Showing AWS GUI Components after file is uploaded.
    display(awsProgressRefreshRateSlider)
    display(awsUploadButton)

In [ ]:
## File upload callback function and button ##
def selectData2upload(imageData): 
    # Showing AWS GUI Components after file is uploaded.
    display(awsUploadButton2)

## Save user input to .csv, and input data to.txt

In [ ]:
def makecsvfile(inputFileName, m):
  with open(inputFileName,"w") as csvfile: 
      writer = csv.writer(csvfile)
      # #write columns_name
      # writer.writerow(["Month","sweaters: (United States)"])
      #write rows
      # m = str(m)
      # s = int(s)
      # writer.writerows([[m[:-3],s]])
      writer.writerows([[str(m)[:-3]]])
     


In [ ]:
def maketxtfile(inputTxtFileName, m,s):
  with open(inputTxtFileName,"w+") as file1: 
      writer = csv.writer(file1)
      m = str(m)
      s = int(s)
      writer.writerows([[m[:-3],s]])

### Creating GUI Components ###



In [ ]:
global hashKeyList
hashKeyList = []

In [ ]:
# for user to get the prediction

def UserDashBoard():
    # Allows the buttons to be accessed globally: Necessary
    # since some callback functions are dependent on these
    # widgets.
    global awsUploadButton
    global awsProgressRefreshRateSlider
    global inputImageFileName
    global resultsDataFileName
    global dropdownID
    global suffix
    global id

    # AWS Image Upload Button.
    awsUploadButton = widgets.Button(description="Get Prediction")
    awsUploadButton.on_click(awsImageUpload)
    
    # AWS Progress Refresh Rate Selector.
    awsProgressRefreshRateSlider = widgets.FloatSlider(description='RefreshRate:', min=0, max=1.0, value=0.5)
    
    # make file name
    now = datetime.now().time()
    # print("Current time is:")
    # print(str(now).encode('utf-8'))
    # print("random hash of current time")
    suffix = hashlib.sha224(str(now).encode('utf-8')).hexdigest()
    inputImageFileName = str(mainInputImageFileName[:-4] + "_" + suffix + ".csv")
    # print("Current input file name is", inputImageFileName)
    resultsDataFileName = str(mainresultsDataFileName[:-4] + "_" + suffix + ".txt")
    hashKeyList.append(resultsDataFileName)
    

    # Display GUI.
    # Note: If new image is being uploaded, this block needs to be
    # run again.

    display("Click 'Enter' when finished selecting")
    # selectimage2uploadButton = widgets.interact(makecsvfile, inputFileName = inputImageFileName, s=widgets.IntSlider(description='Sweaters:',min=0, max=200, step=1, value=0), 
    #                                     m=widgets.IntSlider(description='data:',min=0, max=200, step=1, value=0))
    # user selected Month
    selectimage2uploadButton = widgets.interact(makecsvfile, inputFileName = inputImageFileName, m = widgets.DatePicker(description='Date:',disabled=False))
    
    # Specifying which function to run whenever an image is uploaded.
    selectimage2upload(str(folderName + "/" + inputImageFileName))
    display(selectimage2uploadButton)

In [ ]:
# for manager to input data

def managerDashBoard():
    # Allows the buttons to be accessed globally: Necessary
    # since some callback functions are dependent on these
    # widgets.
    global awsUploadButton2
    global awsProgressRefreshRateSlider

    # AWS Image Upload Button.
    awsUploadButton2 = widgets.Button(description="Upload to AWS")
    awsUploadButton2.on_click(awsDataUpload)
    

    # Display GUI.
    # Note: If new image is being uploaded, this block needs to be
    # run again.

    display("Click 'Enter' when finished selecting")
    # selectimage2uploadButton = widgets.interact(makecsvfile, inputFileName = inputImageFileName, s=widgets.IntSlider(description='Sweaters:',min=0, max=200, step=1, value=0), 
    #                                     m=widgets.IntSlider(description='data:',min=0, max=200, step=1, value=0))
    # select Month and data
    selecData2uploadButton = widgets.interact(maketxtfile, inputTxtFileName = inputDataFileName, s=widgets.IntSlider(description='Sweaters:',min=0, max=200, step=1, value=0), 
                                m = widgets.DatePicker(description='Date:',disabled=False))
    
    # Specifying which function to run whenever an image is uploaded.
    selectData2upload(str(folderName + "/" + inputDataFileName))
    display(selecData2uploadButton)

In [ ]:
managerDashBoard()

"Click 'Enter' when finished selecting"

interactive(children=(Text(value='newData.csv', description='inputTxtFileName'), DatePicker(value=None, descri…

Button(description='Upload to AWS', style=ButtonStyle())

<function __main__.maketxtfile>